In [18]:
#Standalone lines, delete table to re-perform import, close connection when needed, free memory
import gc
c.execute("Drop table if exists Businesses")
c.execute("Drop table if exists Checkins")
c.execute("Drop table if exists Users")
c.execute("Drop table if exists Tips")
c.execute("Drop table if exists Reviews")
c.execute("Drop table if exists LV_Businesses")
c.execute("Drop table if exists Joined")
c.execute("Drop table if exists LV_Reviews")

In [8]:
import pandas as pd
import json
import sqlite3
from sqlite3 import Error

In [68]:
#conn.close()

In [19]:
gc.collect()

44

In [20]:
# Paths to unprocessed dataset files

# prefix = "C:\\Users\\pajuc\\Documents\\yelp\\data\\"
prefix = '../../data/'

filePaths = {
    "reviews": prefix + 'yelp_academic_dataset_review.json',
    "businesses": prefix + 'yelp_academic_dataset_business.json',
    "checkins": prefix + 'yelp_academic_dataset_checkin.json',
    "tips": prefix + 'yelp_academic_dataset_tip.json',
    "users": prefix + 'yelp_academic_dataset_user.json',
    "db": prefix + 'yelp.db'
}

In [21]:
%%time

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn

# fullyelp.db created and used instead of original yelp.db for troubleshooting
db_file = filePaths['db']
conn = create_connection(db_file=db_file)

c = conn.cursor()

2.6.0
Wall time: 12.9 ms


In [22]:
%%time
#Loads business table, entire JSON fits in memory
dfBusinesses = pd.read_json(
    filePaths["businesses"], 
    lines=True
)

# This is a little brute force to get these dicts into SQL text. There are wrappers to maintain the struct if needed...
# Right now they will just be a large string. Since they are structured it may be easier to make some functions to
# extract info. TBD

dfBusinesses['attributes'] = dfBusinesses['attributes'].apply(json.dumps)
dfBusinesses['hours'] = dfBusinesses['hours'].apply(json.dumps)
dfBusinesses.to_sql('Businesses', con=conn)

Wall time: 7.02 s


In [23]:
%%time
#Loads checkins table
dfCheckins = pd.read_json(
    filePaths["checkins"], 
    lines=True
)
dfCheckins.to_sql('Checkins', con=conn)

Wall time: 5.99 s


In [24]:
%%time
#Loads tips table
dfTips = pd.read_json(
    filePaths["tips"], 
    lines=True
)
dfTips.to_sql('Tips', con=conn)

Wall time: 14 s


In [25]:
%%time
# Loads users table
# There is a bug in pandas 1.1.3 (et al) requiring nrows to be greater than # lines in file
# This was causing issues with chunking, but excess nrows fixed it.
# I read it was solved in 1.1.4 but didn't bother trying

for df in pd.read_json(
    filePaths["users"], 
    lines=True,
    orient='records',
    chunksize=100000,
    nrows=234236523
):
    new_users = pd.DataFrame(df)
    new_users.to_sql('Users', con=conn, if_exists='append')

Wall time: 1min 11s


In [26]:
%%time
# Loads reviews table
for df in pd.read_json(
    filePaths["reviews"], 
    lines=True,
    orient='records',
    chunksize=100000, # Dataframe size
    nrows=4234236234234
):
    new_reviews = pd.DataFrame(df)
    new_reviews.to_sql('Reviews', con=conn, if_exists='append')

Wall time: 2min 59s


In [33]:
## Requires exported json from DataReduction_Businesses (Las Vegas businesses subset)
import os
# vegas = pd.read_json(os.path.abspath('') + "Preprocessing/Data Reduction/cleaned_restaurants.json")
vegas = pd.read_json("../Data Reduction/cleaned_restaurants.json")

In [34]:
vegas.head()

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,9595 W Tropicana Ave,Las Vegas,NV,89147,36.099738,-115.301568,2.5,15,0,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...","Mexican, Restaurants, Fast Food",None
1,1,fhNf_sg-XzZ3e7HEVGuOZg,Meat Chix And Wieners,6530 S Decatur Blvd,Las Vegas,NV,89118,36.071196,-115.207463,3.0,10,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Burgers, Restaurants, American (Traditional), ...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
2,3,MTx-Zdl_KcU_z9G832XAjg,Taco Bell,8033 N Durango Dr,Las Vegas,NV,89131,36.307495,-115.279059,2.5,37,1,"{'OutdoorSeating': 'False', 'Alcohol': 'u'none...","Fast Food, Restaurants",None
3,6,upSPwM6Fr6RHAhwagnleow,Manantial De Salud,4254 E Charleston Blvd,Las Vegas,NV,89104,36.159221,-115.082607,5.0,3,1,"{'BusinessAcceptsBitcoin': 'False', 'BikeParki...","Specialty Food, Health Markets, Food, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:30',..."
4,7,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [35]:
%%time
#c.execute("Drop table if exists LV_Businesses")
vegas['attributes'] = vegas['attributes'].apply(json.dumps)
vegas['hours'] = vegas['hours'].apply(json.dumps)
vegas.to_sql('LV_Businesses', con=conn)

Wall time: 120 ms


In [36]:
%%time
# Creates dataframe joining reviews to LV Businesses
# 2GB in-memory
join_query =  pd.read_sql_query("Select * from Reviews Join LV_Businesses on Reviews.business_id = LV_Businesses.business_id", conn)

Wall time: 6min 4s


In [37]:
join_query.head()

,index,review_id,user_id,business_id,stars,useful,funny,cool,text,date,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,16,bVTjZgRNq8ToxzvtiVrqMA,2hRe26HSCAWbFRn5WChK-Q,d4qwVw4PcN-_2mK2o1Ro1g,1,0,0,0,10pm on a super bowl Sunday and they're alread...,2015-02-02 06:28:00,...,NV,89169,36.109706,-115.154021,3.5,1120,1,"{""RestaurantsTakeOut"": ""True"", ""BusinessAccept...","Mexican, Restaurants","{""Monday"": ""11:0-22:0"", ""Tuesday"": ""11:0-22:0""..."
1,21,TeX0eOIeZb_wpo86rCedXQ,RR-2nouBn408e3djxC470g,d4qwVw4PcN-_2mK2o1Ro1g,5,0,0,0,Holy heck this place is amazing. I love their ...,2017-02-09 04:25:03,...,NV,89169,36.109706,-115.154021,3.5,1120,1,"{""RestaurantsTakeOut"": ""True"", ""BusinessAccept...","Mexican, Restaurants","{""Monday"": ""11:0-22:0"", ""Tuesday"": ""11:0-22:0""..."
2,32,NHQWhZve7UEaGLt7IwAyDw,H3wPBzQCKCKiHcV7GZK4kg,LkMtMHVetws5_7QfRjPtlg,2,5,3,4,This actually used to be one of my favorite ho...,2011-01-09 00:09:50,...,NV,89109,36.136257,-115.151253,2.5,944,0,"{""NoiseLevel"": ""u'average'"", ""RestaurantsDeliv...","Casinos, Nightlife, Restaurants, Hotels & Trav...","{""Monday"": ""6:0-6:0"", ""Tuesday"": ""6:0-6:0"", ""W..."
3,40,TkcxD1A4N_9kRZwGhUjDKA,bgG0jJyUXta_kczGrQW2Rw,7EZ4Eu7YJ1ltRCC5jXFJrQ,1,1,0,0,There wasn't one specific thing that made this...,2013-04-10 16:19:12,...,NV,89109,36.124829,-115.171991,2.5,824,1,"{""NoiseLevel"": ""u'average'"", ""RestaurantsDeliv...","Asian Fusion, Event Planning & Services, Resta...","{""Monday"": ""7:0-22:0"", ""Tuesday"": ""7:0-22:0"", ..."
4,46,8qdikDQ0bkJj7QZa2IdvrQ,7MXunAzrmnbBd0XQR33zaA,nJcyfIRGbI31EGLzillWYg,1,0,0,0,A margarita in clear plastic dickie cup??? Wtf...,2015-01-18 16:12:27,...,NV,89109,36.137112,-115.163311,3.0,448,1,"{""Ambience"": ""{'touristy': True, 'hipster': Fa...","Breakfast & Brunch, Nightlife, Bars, American ...","{""Monday"": ""7:0-0:0"", ""Tuesday"": ""7:0-0:0"", ""W..."


In [38]:
join_query.size

21615620

In [39]:
join_query.columns

Index(['index', 'review_id', 'user_id', 'business_id', 'stars', 'useful',
       'funny', 'cool', 'text', 'date', 'level_0', 'index', 'business_id',
       'name', 'address', 'city', 'state', 'postal_code', 'latitude',
       'longitude', 'stars', 'review_count', 'is_open', 'attributes',
       'categories', 'hours'],
      dtype='object')

In [40]:
# Removes extraneous garbage columns
# This can be taken care of with earlier import/exports but done here to avoid re-creating data tables
join_query = join_query.drop(columns=['index', 'level_0'])

In [41]:
# Pandas read_query keeps join columns unlike regular SQL join
# Removing to allow table creation in database
join_query = join_query.loc[:,~join_query.columns.duplicated()]

In [42]:
join_query.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [43]:
join_query.to_sql("LV_Reviews", conn, if_exists="replace", index=False)

In [44]:
gc.collect()
conn.close() 